<a href="https://colab.research.google.com/github/BrenoAV/ChatBot_Pizzaria_Super_Legal/blob/main/ChatBot_Pizzaria_Super_Legal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 4.1 MB/s eta 0:00:00


In [12]:
from datetime import datetime
import pytz
import numpy as np
from numpy.linalg import norm
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
import textwrap
from IPython.display import display, Markdown, display_markdown

GEMINI_KEY = userdata.get("GEMINI_KEY")

genai.configure(api_key=GEMINI_KEY)

date = datetime.now(pytz.timezone('America/Recife'))

# Funções Auxiliares

In [3]:
def to_markdown(text: str) -> Markdown:
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Lista todos os modelos de geração de conteúdo

In [4]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


# Documentos

In [5]:
doc_menu = """Cardápio da pizzaria super legal com todos os valores de pizzas e bebidas disponíveis na pizzaria.

# Pizzas Tradicionais

Preços das pizzas tradicionais:
  - R$ 20,00 (Pequena - P)
  - R$ 30,00 (Média - M)
  - R$ 40,00 (Grande - G)

01. Mussarela - Molho, mussarela, tomate, orégano e azeitona.
02. Dois queijos - Molho, mussarela, catupiry, orégano e azeitona.
03. Frango - Molho, frango, mussarela, milho, orégano e azeitona.
04. Vegetariana - Molho, milho, ervilha, cebola, tomate, palmito, brócolis, mussarela, orégano e azeitona.
05. Nordestina - Molho, carne cebola, mussarela, orégano e azeitona.
06. Calabresa - Molho, calabresa, bacon, orégano e azeitona.
07. Portuguesa - Molho, presunto, cebola, ervilha, ovos, mussarela, orégano e azeitona.
08. Baiana - Molho, calabresa, ovos, pimenta, cebola, mussarela, orégano e azeitona.

# Pizzas Especiais

Preços das pizzas especiais:
  - R$ 25,00 (Pequena - P)
  - R$ 35,00 (Média - M)
  - R$ 45,00 (Grande - G)

01. Camarão - Molho, camarão, mussarela, catupiry, orégano e azeitona.
02. Peru - Molho, peito de peru, mussarela, catupiry, orégano e azeitona.
03. Brasileira - Molho, peito de peru, parmesão, creme cheese, mussarela, bacon, orégano e azeitona.

# Pizzas Doces

Preços das pizzas doces:
  - R$ 25,00 (Pequena - P)
  - R$ 35,00 (Média - M)
  - R$ 45,00 (Grande - G)

01. Sonho de Valsa - Bombons, creme de leite, leite condensado e cereja.
02. Brigadeiro - Chocolate, granulado, leite condensado e cereja.
03. Chocovalsa - Chocolate, bombons, leite condensado, creme de leite e cereja.

# Bebidas

01. Água (R$ 2,00)
02. Refrigerante Lata (R$ 5,00)
03. Refrigerante 1L (R$ 8,00)
04. Refrigerante 2L (R$ 12,00)
05. Suco 600 mL (R$ 5,00)

# Promoções das Pizzas

Promoção: Nas sexta-feiras, sábado e domingo as pizzas tradicionais de Frango e Calabresa estão com um desconto de 30% para todos os tamanhos

"""

In [6]:
generation_config = {
    "temperature": 0,
    "candidate_count": 1
}

gen_model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                                  generation_config=generation_config,
                                  system_instruction=["Aja como um antendente expert de uma pizzaria",
                                                      f"A data de hoje é {date.date().strftime('%A')}",
                                                      "Não forneça nenhuma informação que não esteja no contexto da pizzaria, por exemplo, inventar novos tamanhos de pizzas ou inventar novos sabores de pizza.",
                                                      "Se o cliente não informar o tamanho de todas as pizzas, não finalize o pedido.",
                                                      "Não invente nada que não esteja disponível na pizzaria, informe para o cliente gentilmente que não existe o requisito.",
                                                      "Aplique os possíveis descontos das promoções da pizzaria e informe na resposta final o que teve desconto (se aplicável).",
                                                      "Quando finalizar o pedido simplesmente retorne na última linha: fim do pedido"
                                                      ]
)


In [7]:
chatbot_pizzaria = gen_model.start_chat(history=[])

pedido = input("Por favor, informe seu pedido (digite 'sair' para finalizar o antendimento)\nComando: ")
fim_do_pedido = None

while True:
    prompt = ("O contexto da pizzaria é:\n\n"
              "---"
              f"{doc_menu}\n\n"
              "---"
              f"Cliente: {pedido}\n"
              )
    response = chatbot_pizzaria.send_message(prompt)
    print("ChatBot:\n\n", response.text)
    print("-"*60)
    fim_do_pedido = response.text.split("\n")[-2]
    if fim_do_pedido == "fim do pedido":
        break
    pedido = input("Comando: ")
    print("-"*60)
    if pedido.lower().strip() == "sair":
        break


Por favor, informe seu pedido (digite 'sair' para finalizar o antendimento)
Comando: Uma pizza grande de calabresa
ChatBot:

 Certo, uma pizza grande de calabresa. Qual o sabor da próxima pizza? 

------------------------------------------------------------
Comando: Somente isso
------------------------------------------------------------
ChatBot:

 Certo, uma pizza grande de calabresa por R$ 28,00 (com desconto de 30%).

fim do pedido

------------------------------------------------------------


# Áudios Prompts

In [10]:
audio_file = genai.upload_file(path="pedido1.m4a")
prompt = ("O contexto da pizzaria é:\n"
         f"{retrieval}\n"
          "Forneça o resumo do pedido discriminado. Baseado no áudio.")

response = gen_model.generate_content([prompt, audio_file])

print(response.text)

FileNotFoundError: [Errno 2] No such file or directory: 'pedido1.m4a'